In [405]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from IPython.core.debugger import Tracer

%matplotlib inline

global nxy # needed to smoothen and draw a single line
nxy = [1200,0,0,0]

global pxy # needed to smoothen and draw a single line
pxy = [1200,0,0,0]

global limitsc #Used for challenge video to smoothen the lines

limitsc = [300,0,590,460,740,450,1070,0]
    
global limitsr #did not use this

limitsr = [100,0,420,330,600,330,900,0]

global svideo #determines if video is challenge video
svideo = 1




In [406]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def assign_nvalue (x1,y1,xy):

    if (x1 < xy[0]):
        xy[0] = x1
        xy[1] = y1
    elif (x1 > xy[2]):
        xy[2] = int((x1 + xy[2])/2)
        xy[3] = int((y1 + xy[3])/2)
    return xy

def assign_pvalue (x1,y1,xy):

    if (x1 < xy[0]):
        xy[0] = int(0.8*x1 + 0.2*xy[0])
        xy[1] = y1
    elif (x1 > xy[2]):
        xy[2] = int((x1 + xy[2])/2)
        xy[3] = int((y1 + xy[3])/2)
    return xy

                        
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if lines is None: 
        return
    
    global nxy 
    #nxy = [1200,0,0,0]
    global pxy 
    #pxy = [1200,0,0,0]

    global limitsc,limitsr,svideo
    
    nl = len(lines)
    slope = list ()
    negslope = list() 
    i = 0
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            s = (y2-y1) / (x2-x1)
            if (s < 0):
                if (s < -0.577 and s > -11.5):
                    negslope.append (s)
                    nxy = assign_nvalue (x1,y1,nxy)                        
                    nxy = assign_nvalue (x2,y2,nxy)
            else:
                if (s > 0.577 and s < 11.5):
                    slope.append (s)
                    pxy = assign_pvalue (x1,y1,pxy)
                    pxy = assign_pvalue (x2,y2,pxy)
    
    #medianslope = np.median(np.asarray(slope),axis=0)
    #mediannegslope = np.median (np.asarray(negslope), axis=0)
  
    # correct for variance 
    
    if (svideo == 2):
        if (nxy[0] < limitsc[0]):
            nxy[0] = limitsc[0]
        if (nxy[2] > limitsc[2]):
            nxy[2] = limitsc[2]
            nxy[3] = limitsc[3]

        if (pxy[0] < limitsc[4]):
            pxy[0] = limitsc[4]
            pxy[1] = limitsc[5]
        if (pxy[2] > limitsc[6]):
            pxy[2] = limitsc[6]
    
    #end of correction    
    
    #draw the line based on about computation
    cv2.line (img, (nxy[0],nxy[1]),(nxy[2],nxy[3]),color,thickness)
    cv2.line (img, (pxy[0],pxy[1]),(pxy[2],pxy[3]),color,thickness)
    
 
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
                            slope = (y2-y1) / (x2-x1)"""
    

    #    vertices = np.array([[(150,539),(440, 325), (550, 325), (900,539)]], dtype=np.int32)
    global svideo
    
    if (svideo == 1):
        vertices = np.array([[(100,539),(420, 330), (600, 330), (900,539)]], dtype=np.int32)
    else:
        vertices = np.array([[(225,665),(560, 460), (740, 450), (1070,630)]], dtype=np.int32)
    
    masked_edges = region_of_interest(img,vertices)
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_len, max_line_gap)

    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    masked_edges = region_of_interest(line_img,vertices)
    
    return masked_edges

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

#Dr.Yadav's code for colorMask
def color_mask(hsv,low,high):
    mask = cv2.inRange(hsv, low, high)
    return mask



In [407]:

def process_image(image):
    
    
    gray = grayscale(image)
        
    blur = gaussian_blur(gray, 5)
    
    
    # Following mask was taken from Dr. Yadav in Medium
    # https://github.com/vxy10/p4-Advanced_Lane_Findings/blob/master/main_lane_markings_fin2.ipynb
    
    yellow_hsv_low  = np.array([ 0, 100, 100])
    yellow_hsv_high = np.array([ 50, 255, 255])

    white_hsv_low  = np.array([  20,   0,   180])
    white_hsv_high = np.array([ 255,  80, 255])
    
    cblur = gaussian_blur(image,5)
    
    image_HSV = cv2.cvtColor(cblur,cv2.COLOR_RGB2HSV)

    mask_yellow = color_mask(image_HSV,yellow_hsv_low,yellow_hsv_high)
    mask_white = color_mask(image_HSV,white_hsv_low,white_hsv_high)
    mask_lane = cv2.bitwise_or(mask_yellow,mask_white) 
    
    #END of Dr.Yadav's Code

    #Compute canny edges
    
    edges = canny(blur, 40, 120)

    #combining Dr.Yadav mask with canny image to get better lines
    
    newedges = cv2.bitwise_or(edges,mask_lane)
    
    lines_edges = hough_lines(newedges, 1, np.pi/180, 5, 30, 10)
 
    # Draw the lines on the edge image
    final_image = weighted_img(lines_edges, image)
    
    return final_image

In [408]:
#this code is only for testing

#image = mpimg.imread('test_images/solidWhiteRight.jpg')
#svideo = 1

#final_img = process_image(image)
#plt.imshow(final_img)

In [409]:
import imageio
imageio.plugins.ffmpeg.download()

In [410]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML


white_output1 = 'whiteRightOutput.mp4'
svideo = 1
nxy = [1200,0,0,0]
pxy = [1200,0,0,0]

clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output1, audio=False)


[MoviePy] >>>> Building video whiteRightOutput.mp4
[MoviePy] Writing video whiteRightOutput.mp4


100%|███████████████████████████████████████▊| 221/222 [00:05<00:00, 42.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: whiteRightOutput.mp4 

Wall time: 5.7 s


In [ ]:

white_output2 = 'yellowLeftOutput.mp4'
svideo = 1
nxy = [1200,0,0,0]
pxy = [1200,0,0,0]

clip1 = VideoFileClip("solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output2, audio=False)


In [ ]:

white_output3 = 'challengeOutput.mp4'
svideo = 2
nxy = [1200,0,0,0]
pxy = [1200,0,0,0]
clip1 = VideoFileClip("challenge.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output3, audio=False)


In [403]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output1))

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output2))

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output3))